# Zonal Exchange Network
ZEN（Zonal Exchange Network） 是一个原创的水文模型，用于模拟流域内的水文过程。ZEN使用神经网络来描述产汇流过程。

In [32]:
# 自动加载模块
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. 数据处理

In [33]:
from hydronetwork.data.camels_us import load_timeseries
from hydronetwork.dataset.preprocessing import split_timeseries
from hydronetwork.dataset.dataset import get_dataset

lookback = 365
horizon = 7
timeseries = load_timeseries(gauge_id='13023000', unit="mm/day")
timeseries.columns

Index(['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)',
       'streamflow'],
      dtype='object', name='timeseries_type')

对['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)']进行归一化

In [34]:
from hydronetwork.dataset.preprocessing import normalize

timeseries_need_normalize = timeseries[['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)']]
timeseries_normalized = normalize(timeseries_need_normalize)
timeseries = timeseries.drop(columns=['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'])
timeseries = timeseries.join(timeseries_normalized)

构建数据集

In [35]:
train_data, test_data = split_timeseries(timeseries, split_list=[0.8, 0.2])
train_dataset = get_dataset(train_data, 
                            lookback=lookback, 
                            horizon=horizon, 
                            batch_size=2048, 
                            features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
                            target="streamflow"
                            )
test_dataset = get_dataset(test_data,
                            lookback=lookback,
                            horizon=horizon,
                            batch_size=2048,
                            features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
                            target="streamflow"
                            )

## 2. 模型构建

In [36]:
from hydronetwork.model import ZonalExchangeNet

model = ZonalExchangeNet(m=3,
                         n=64,
                         n_mix_steps=3,
                         water_capacity_max=120,
                         layer_units=[16, 8, 1],
                         horizon=horizon, )

## 3. 模型训练

In [ ]:
from hydronetwork.train import WarmupExponentialDecay, callback_for_features_selection
from hydronetwork.evaluate.metric import rmse, nse
from keras.src.optimizers import AdamW

model.compile(optimizer=AdamW(learning_rate=WarmupExponentialDecay(dataset_length=len(train_dataset),
                                                                   initial_learning_rate=1e-4, )),
              loss=nse,
              metrics=[rmse]
              )
model.fit(train_dataset,
          epochs=60,
          verbose=1,
          shuffle=True,
          validation_data=test_dataset,
          callbacks=callback_for_features_selection())

Epoch 1/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 195s 39s/step - loss: 16.1970 - rmse: 5.9008 - val_loss: 61.5527 - val_rmse: 5.6272
Epoch 2/60
1/5 ━━━━━━━━━━━━━━━━━━━━ 2:30 38s/step - loss: 12.2038 - rmse: 3.7291